In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

pd.set_option('display.max_rows', 1500)

In [62]:
timing_df = pd.read_csv(os.path.join('data_sim', 'tree_timing_logger_1thread.txt'), sep=',', header=0).iloc[:,:-1]

In [63]:
timing_df['delta_t_ns'] =  timing_df['high_precision_timestamp']-timing_df['high_precision_timestamp'].shift(1)

for idx, category in enumerate(timing_df['category'].unique()):
    filtered_df = timing_df.loc[timing_df['category']==category]
    filtered_df['delta_t_category_ns'] = filtered_df['high_precision_timestamp']-filtered_df['high_precision_timestamp'].shift(1)
    
    if idx==0:
        sum_df = filtered_df
    else:
        sum_df = pd.concat([sum_df, filtered_df])
    
    
timing_df['delta_t_category_ns'] = sum_df['delta_t_category_ns']

    


/home/etienne/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [64]:
summary_dict={}
for idx_category, category in enumerate(timing_df['category'].unique()):
    filtered_df = timing_df.loc[timing_df['category']==category]
    print(category)
    
    for idx_t0_internal, t0_internal in enumerate(filtered_df['t0_internal'].unique()):
        
        
        if idx_t0_internal == 0:
            summary_df = filtered_df.loc[(filtered_df['t0_internal']==t0_internal)][['description', 'category', 'delta_t_category_ns']].reset_index(drop=True)
        else:
            resetted_idx_df = filtered_df.loc[(filtered_df['t0_internal']==t0_internal)]['delta_t_category_ns'].reset_index(drop=True)
            summary_df['delta_t_category_ns_'+str(idx_t0_internal)] = resetted_idx_df
    
    summary_dict[category] = {'raw': 0, 'metrics': 0}
    summary_dict[category]['raw_df'] = summary_df
    
    metrics_df = summary_df.iloc[:,:2]
    metrics_df['mean'] = summary_df.iloc[:,2:].mean(axis=1)
    metrics_df['median'] = summary_df.iloc[:,2:].median(axis=1)
    metrics_df['q1'] = summary_df.iloc[:,2:].quantile(q=0.25, axis=1)
    metrics_df['q3'] = summary_df.iloc[:,2:].quantile(q=0.75, axis=1)
    metrics_df['percentage'] =  metrics_df['median']/sum(metrics_df['median'])
    
    
    
    summary_dict[category]['metrics_df'] = metrics_df
    

build_new_tree
grow_tree
add_depth_level
add_node


In [65]:
with pd.ExcelWriter('timing_1_thread_metrics.xlsx', engine='xlsxwriter') as excel_writer:       
    for key in summary_dict:
        display(summary_dict[key]['metrics_df'])
        summary_dict[key]['metrics_df'].to_excel(excel_writer, sheet_name=key)

,description,category,mean,median,q1,q3,percentage
0,start of build new tree,build_new_tree,25049.555944,26370.0,24422.75,26850.75,0.791583
1,start of build new tree after reset dynamics,build_new_tree,956.285215,746.0,659.50,1413.00,0.022394
2,start of build new tree after init tree,build_new_tree,403.828871,289.0,256.00,593.00,0.008675
3,start of build new tree after grow tree,build_new_tree,6812.393939,5804.0,5414.00,6876.00,0.174226
4,start of build new tree after convert to rollouts,build_new_tree,112.272727,104.0,86.00,129.00,0.003122


,description,category,mean,median,q1,q3,percentage
0,before add_depth_level(i),grow_tree,26535.759907,27682.0,25997.5,28236.00,0.831703
1,after add_depth_level(i) / before eval_depth_l...,grow_tree,968.562282,661.0,592.0,953.50,0.019860
2,after eval_depth_level(),grow_tree,5.763679,4.0,3.0,6.00,0.000120
3,before add_depth_level(i),grow_tree,4.608847,3.0,3.0,5.00,0.000090
4,after add_depth_level(i) / before eval_depth_l...,grow_tree,944.875437,650.0,578.5,846.50,0.019529
5,after eval_depth_level(),grow_tree,4.989523,3.0,3.0,5.00,0.000090
6,before add_depth_level(i),grow_tree,4.282887,3.0,2.0,4.00,0.000090
7,after add_depth_level(i) / before eval_depth_l...,grow_tree,895.079254,641.0,571.0,798.50,0.019259
8,after eval_depth_level(),grow_tree,4.418415,3.0,3.0,4.00,0.000090
9,before add_depth_level(i),grow_tree,3.785548,3.0,2.0,3.00,0.000090


,description,category,mean,median,q1,q3,percentage
0,before parallel processing of nodes,add_depth_level,26547.764569,27693.5,26006.50,28243.75,0.832224
1,after parallel processing of nodes / before re...,add_depth_level,923.686845,631.0,567.00,925.50,0.018962
2,after resetting dynamics,add_depth_level,32.016298,23.0,20.00,34.00,0.000691
3,before parallel processing of nodes,add_depth_level,21.542491,14.0,12.00,23.50,0.000421
4,after parallel processing of nodes / before re...,add_depth_level,915.658906,632.0,563.00,815.50,0.018992
5,after resetting dynamics,add_depth_level,19.236321,11.0,9.00,21.00,0.000331
6,before parallel processing of nodes,add_depth_level,18.863795,12.0,11.00,18.00,0.000361
7,after parallel processing of nodes / before re...,add_depth_level,869.645688,623.5,557.00,772.00,0.018737
8,after resetting dynamics,add_depth_level,16.368298,10.0,9.00,17.00,0.000301
9,before parallel processing of nodes,add_depth_level,16.885781,12.0,10.00,15.75,0.000361


,description,category,mean,median,q1,q3,percentage
0,beginning add_node,add_node,26612.631702,27752.0,26073.25,28313.00,0.836786
1,after node_dynamics->reset.,add_node,24.973225,19.0,16.50,27.00,0.000573
2,after getting samples and sigma inv.,add_node,7.735739,5.0,5.00,10.00,0.000151
3,after stepping dynamics.,add_node,5.514552,4.0,3.00,9.00,0.000121
4,after appending new node to tree.,add_node,18.835856,13.0,12.00,21.50,0.000392
5,beginning add_node,add_node,13.775320,10.0,9.00,17.00,0.000302
6,after node_dynamics->reset.,add_node,5.912689,4.0,3.00,8.00,0.000121
7,after getting samples and sigma inv.,add_node,9.467986,7.0,6.00,13.00,0.000211
8,after stepping dynamics.,add_node,4.792782,3.0,3.00,7.50,0.000090
9,after appending new node to tree.,add_node,10.847497,7.0,6.00,14.00,0.000211
